In [2]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

# Load images from directory
def load_images_from_folder(folder, img_size=(100, 100)):
    X, y = [], []
    for filename in os.listdir(folder):
        if filename.endswith(".jpg"):
            label = filename.split("-")[0]  # Extract person ID
            img_path = os.path.join(folder, filename)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            if img is not None:
                img = cv2.resize(img, img_size)
                X.append(img)
                y.append(label)
    return np.array(X), np.array(y)

# Load data
folder_path = "originalimages"  # Update this path
X, y = load_images_from_folder(folder_path)

# Normalize and reshape
X = X.astype("float32") / 255.0
X = X.reshape(-1, 100, 100, 1)

# Encode labels
le = LabelEncoder()
y_encoded = le.fit_transform(y)
y_cat = to_categorical(y_encoded)

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y_cat, test_size=0.2, stratify=y_cat, random_state=42
)

ImportError: Traceback (most recent call last):
  File "C:\Users\abidi\AppData\Roaming\Python\Python312\site-packages\tensorflow\python\pywrap_tensorflow.py", line 73, in <module>
    from tensorflow.python._pywrap_tensorflow_internal import *
ImportError: DLL load failed while importing _pywrap_tensorflow_internal: Une routine d’initialisation d’une bibliothèque de liens dynamiques (DLL) a échoué.


Failed to load the native TensorFlow runtime.
See https://www.tensorflow.org/install/errors for some common causes and solutions.
If you need help, create an issue at https://github.com/tensorflow/tensorflow/issues and include the entire stack trace above this error message.

In [ ]:
from tensorflow.keras import layers, models

# CNN feature extractor
def build_cnn_feature_extractor(input_shape):
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.Flatten())  # Keep only features, no Dense output here
    return model

cnn_model = build_cnn_feature_extractor((100, 100, 1))

# Extract CNN features
X_train_feat = cnn_model.predict(X_train)
X_test_feat = cnn_model.predict(X_test)

In [ ]:
from sklearn.decomposition import PCA

# Reduce dimensions
pca = PCA(n_components=100)
X_train_pca = pca.fit_transform(X_train_feat)
X_test_pca = pca.transform(X_test_feat)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

mlp_model = Sequential([
    Dense(128, activation='relu', input_shape=(100,)),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(y_cat.shape[1], activation='softmax')
])

mlp_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history = mlp_model.fit(
    X_train_pca, y_train,
    validation_data=(X_test_pca, y_test),
    epochs=25,
    batch_size=32
)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 5))

# Loss
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label="Training Loss")
plt.plot(history.history['val_loss'], label="Validation Loss")
plt.title("Loss Curve")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()

# Accuracy
plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label="Training Accuracy")
plt.plot(history.history['val_accuracy'], label="Validation Accuracy")
plt.title("Accuracy Curve")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
def predict_face(img_path):
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        print("❌ Image not found.")
        return
    img = cv2.resize(img, (100, 100)).astype("float32") / 255.0
    img = img.reshape(1, 100, 100, 1)

    features = cnn_model.predict(img)
    reduced = pca.transform(features)
    prediction = mlp_model.predict(reduced)
    predicted_label = np.argmax(prediction)
    identity = le.inverse_transform([predicted_label])[0]

    plt.imshow(img.reshape(100, 100), cmap='gray')
    plt.title(f"Predicted ID: {identity}")
    plt.axis('off')
    plt.show()

    return identity

# Example usage
predict_face("/content/drive/MyDrive/FEI_Faces/52-08.jpg")
